In [25]:
test_loc = "/home/speechlab/Downloads/test_set_new.txt"
repo_name = "MY_st1_training_shreya_fixed_27_march_labled-decoded_level2"
test_result_file = "Student2_new_test_result_modified"

In [14]:
import soundfile as sf
import torch
import numpy as np
ETLT = {}
ETLT["test"] = {}
ETLT["test"]["text"] = list()
ETLT["test"]["file"] = list()
ETLT["test"]["audio"] = list()
print(ETLT)


test_f = open(test_loc,"r")
for line in test_f.readlines():
    #data = line.split("\t",1)
    ETLT["test"]["text"].append(line.split("\t",1)[1].rstrip("\n").lower())
    ETLT["test"]["file"].append(line.split("\t",1)[1].replace("\n",""))
    #audio = sf.read(data[0])
    #print(audio)
    ETLT["test"]["audio"].append(sf.read(line.split("\t",1)[0].replace("\n",""))[0])
    
test_f.close()
print(len(ETLT["test"]["text"]))
data_dict_test = list()
for i in range(len(ETLT["test"]["file"])):
    data_dict_test.append({"text" : ETLT["test"]["text"][i]  , "file" : ETLT["test"]["file"][i] , "audio" : ETLT["test"]["audio"][i]})

{'test': {'text': [], 'file': [], 'audio': []}}
5001


In [15]:
ETLT["test"]

{'text': ['more energy',
  'energy',
  'um more electricity',
  'well if you add more',
  "i don't know",
  "to to wind up the wire more on the rivet and you'll have more power",
  'add up more and the next amount you can get is fifteen',
  'stronger',
  'wire wraps',
  'well how many washers they picked up',
  "you'll get more washers",
  'adding up washers',
  "there's getting more and more washers",
  'well',
  "i don't know",
  'fifteen',
  'probably about sixteen',
  'done',
  'bye marnie',
  "i've been great how have you been",
  "i'm learning about magnets and the pull about magnets and how manecs magnets work",
  "put a bunch of spacers between the magnets um the strength of the magnet magnet's force won't be as strong",
  "it will break some of the force so then um the magnets can't do they can't have strength anymore and they'll just be just all weak",
  'can you repeat that please',
  "i don't really know that",
  "i don't know that yet",
  "e talking to you i'll see ya tomo

In [16]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained(repo_name).cuda()
processor = Wav2Vec2Processor.from_pretrained(repo_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at MY_st1_training_shreya_fixed_27_march_labled-decoded_level2 and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
def prepare_dataset(batch):
       #audio = batch["audio"]
  # batch
   #fs = sf.read(batch["file"])[1]
   # batched output is "un-batched" to ensure mapping is correct
   batch["input_values"] = processor(batch["audio"], sampling_rate=16000).input_values[0]
   batch["input_length"] = len(batch["input_values"])
    
   with processor.as_target_processor():
       batch["labels"] = processor(batch["text"]).input_ids
   #data_processed = {"input_values" : batch["input_values"], "input_length" : batch["input_length"], "labels" : batch["labels"]}
   return batch

In [18]:
data_processed_test = list(map(prepare_dataset,data_dict_test))

In [19]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [20]:
results = []
for itr in range(len(data_processed_test)):
    print(itr)
    results.append(map_to_result(data_processed_test[itr]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [26]:
pred_str = []
original_str = []
for itr in range(len(results)):
    pred_str.append(results[itr]["pred_str"].replace("[PAD]",""))
    original_str.append(results[itr]["text"])

In [27]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=pred_str, references=original_str)))

Test WER: 0.333


In [29]:
file_f = open(test_result_file,"w")
for x in results:
    file_f.write(str("Original: \t["+ x["text"]+"] \t Pred_str: \t["+ x["pred_str"].replace("[PAD]","")+"] \n"))
    #print("Original: \t[" + x["text"] + "] \t Pred_str: \t[" + x["pred_str"] + "] \n")
file_f.close()